# 标准库对比实现 - TensorFlow/Keras神经网络

## 概述
本notebook使用TensorFlow/Keras框架实现神经网络，与手写实现进行对比分析。我们将探索深度学习框架的优势，包括：
- 更简洁的API
- 更高效的计算
- 更丰富的功能
- 更好的性能优化

In [ ]:
# 导入深度学习框架和其他必要库
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# TensorFlow/Keras导入
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers, models, optimizers, losses, metrics, callbacks
    from tensorflow.keras.utils import to_categorical
    print(f"TensorFlow版本: {tf.__version__}")
    TF_AVAILABLE = True
except ImportError:
    print("TensorFlow未安装，将使用模拟实现")
    TF_AVAILABLE = False

# PyTorch作为备选
try:
    import torch
    import torch.nn as nn
    import torch.optim as optim
    import torch.nn.functional as F
    from torch.utils.data import DataLoader, TensorDataset
    print(f"PyTorch版本: {torch.__version__}")
    PYTORCH_AVAILABLE = True
except ImportError:
    print("PyTorch未安装")
    PYTORCH_AVAILABLE = False

%matplotlib inline

print("库导入完成")

## 1. 数据准备和预处理

In [ ]:
def load_and_preprocess_data():
    """加载和预处理MNIST数据"""
    print("正在加载MNIST数据集...")
    
    # 从sklearn加载MNIST数据
    mnist = fetch_openml('mnist_784', version=1, as_frame=False)
    X, y = mnist.data, mnist.target.astype(int)
    
    print(f"原始数据形状: X={X.shape}, y={y.shape}")
    
    # 数据标准化
    X = X.astype('float32') / 255.0
    
    # 分割数据集
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=0.125, random_state=42, stratify=y_temp
    )  # 0.125 * 0.8 = 0.1 overall
    
    print(f"数据分割:")
    print(f"  训练集: {X_train.shape}")
    print(f"  验证集: {X_val.shape}")
    print(f"  测试集: {X_test.shape}")
    
    # 为不同的框架准备数据
    data = {
        'X_train': X_train, 'X_val': X_val, 'X_test': X_test,
        'y_train': y_train, 'y_val': y_val, 'y_test': y_test
    }
    
    return data

# 加载数据
data = load_and_preprocess_data()

## 2. TensorFlow/Keras实现

In [ ]:
if TF_AVAILABLE:
    def create_tf_model(input_shape=(784,), num_classes=10, hidden_layers=[256, 128]):
        """使用TensorFlow/Keras创建神经网络模型"""
        model = models.Sequential()
        
        # 输入层
        model.add(layers.Input(shape=input_shape))
        
        # 隐藏层
        for i, units in enumerate(hidden_layers):
            model.add(layers.Dense(units, activation='relu', name=f'hidden_{i+1}'))
            model.add(layers.Dropout(0.2, name=f'dropout_{i+1}'))  # 添加Dropout防止过拟合
        
        # 输出层
        model.add(layers.Dense(num_classes, activation='softmax', name='output'))
        
        return model
    
    def compile_and_train_tf_model(model, data, epochs=50, batch_size=64, learning_rate=0.001):
        """编译和训练TensorFlow模型"""
        # 编译模型
        model.compile(
            optimizer=optimizers.Adam(learning_rate=learning_rate),
            loss=losses.SparseCategoricalCrossentropy(),
            metrics=['accuracy']
        )
        
        # 设置回调函数
        callbacks_list = [
            callbacks.EarlyStopping(
                monitor='val_loss',
                patience=10,
                restore_best_weights=True,
                verbose=1
            ),
            callbacks.ReduceLROnPlateau(
                monitor='val_loss',
                factor=0.5,
                patience=5,
                min_lr=1e-7,
                verbose=1
            )
        ]
        
        # 训练模型
        start_time = time.time()
        
        history = model.fit(
            data['X_train'], data['y_train'],
            validation_data=(data['X_val'], data['y_val']),
            epochs=epochs,
            batch_size=batch_size,
            callbacks=callbacks_list,
            verbose=1
        )
        
        training_time = time.time() - start_time
        print(f"训练完成，用时: {training_time:.2f}秒")
        
        return model, history, training_time
    
    def evaluate_tf_model(model, data):
        """评估TensorFlow模型"""
        # 在测试集上评估
        test_loss, test_accuracy = model.evaluate(data['X_test'], data['y_test'], verbose=0)
        
        # 预测
        y_pred_proba = model.predict(data['X_test'])
        y_pred = np.argmax(y_pred_proba, axis=1)
        
        return {
            'test_loss': test_loss,
            'test_accuracy': test_accuracy,
            'y_pred': y_pred,
            'y_pred_proba': y_pred_proba
        }
    
    # 创建和训练TensorFlow模型
    print("=== TensorFlow/Keras实现 ===")
    tf_model = create_tf_model()
    tf_model.summary()
    
    tf_trained_model, tf_history, tf_training_time = compile_and_train_tf_model(tf_model, data)
    tf_results = evaluate_tf_model(tf_trained_model, data)
    
    print(f"\nTensorFlow模型结果:")
    print(f"  测试准确率: {tf_results['test_accuracy']:.4f}")
    print(f"  测试损失: {tf_results['test_loss']:.4f}")
    print(f"  训练时间: {tf_training_time:.2f}秒")
else:
    print("TensorFlow不可用，跳过TensorFlow实现")
    tf_model = None
    tf_history = None
    tf_results = None
    tf_training_time = None

## 3. PyTorch实现

In [ ]:
if PYTORCH_AVAILABLE:
    class PyTorchNN(nn.Module):
        """PyTorch神经网络定义"""
        def __init__(self, input_size=784, hidden_sizes=[256, 128], num_classes=10, dropout_rate=0.2):
            super(PyTorchNN, self).__init__()
            
            layers = []
            
            # 输入层到第一个隐藏层
            layers.append(nn.Linear(input_size, hidden_sizes[0]))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_rate))
            
            # 隐藏层之间
            for i in range(len(hidden_sizes) - 1):
                layers.append(nn.Linear(hidden_sizes[i], hidden_sizes[i+1]))
                layers.append(nn.ReLU())
                layers.append(nn.Dropout(dropout_rate))
            
            # 最后一个隐藏层到输出层
            layers.append(nn.Linear(hidden_sizes[-1], num_classes))
            
            self.network = nn.Sequential(*layers)
        
        def forward(self, x):
            return self.network(x)
    
    def train_pytorch_model(model, data, epochs=50, batch_size=64, learning_rate=0.001):
        """训练PyTorch模型"""
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)
        
        # 准备数据加载器
        train_dataset = TensorDataset(
            torch.FloatTensor(data['X_train']),
            torch.LongTensor(data['y_train'])
        )
        val_dataset = TensorDataset(
            torch.FloatTensor(data['X_val']),
            torch.LongTensor(data['y_val'])
        )
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
        
        # 定义损失函数和优化器
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode='min', factor=0.5, patience=5, verbose=True
        )
        
        # 训练循环
        train_losses = []
        val_losses = []
        train_accuracies = []
        val_accuracies = []
        
        best_val_loss = float('inf')
        patience_counter = 0
        
        start_time = time.time()
        
        for epoch in range(epochs):
            # 训练阶段
            model.train()
            train_loss = 0.0
            train_correct = 0
            train_total = 0
            
            for batch_X, batch_y in train_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                
                train_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                train_total += batch_y.size(0)
                train_correct += (predicted == batch_y).sum().item()
            
            # 验证阶段
            model.eval()
            val_loss = 0.0
            val_correct = 0
            val_total = 0
            
            with torch.no_grad():
                for batch_X, batch_y in val_loader:
                    batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                    outputs = model(batch_X)
                    loss = criterion(outputs, batch_y)
                    
                    val_loss += loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    val_total += batch_y.size(0)
                    val_correct += (predicted == batch_y).sum().item()
            
            # 计算平均损失和准确率
            avg_train_loss = train_loss / len(train_loader)
            avg_val_loss = val_loss / len(val_loader)
            train_accuracy = train_correct / train_total
            val_accuracy = val_correct / val_total
            
            train_losses.append(avg_train_loss)
            val_losses.append(avg_val_loss)
            train_accuracies.append(train_accuracy)
            val_accuracies.append(val_accuracy)
            
            # 学习率调度
            scheduler.step(avg_val_loss)
            
            # 早停
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                patience_counter = 0
                # 保存最佳模型
                best_model_state = model.state_dict().copy()
            else:
                patience_counter += 1
            
            if patience_counter >= 10:
                print(f"早停于epoch {epoch+1}")
                break
            
            if (epoch + 1) % 10 == 0:
                print(f"Epoch {epoch+1}/{epochs}: ")
                print(f"  训练损失: {avg_train_loss:.4f}, 训练准确率: {train_accuracy:.4f}")
                print(f"  验证损失: {avg_val_loss:.4f}, 验证准确率: {val_accuracy:.4f}")
        
        # 加载最佳模型
        model.load_state_dict(best_model_state)
        
        training_time = time.time() - start_time
        print(f"训练完成，用时: {training_time:.2f}秒")
        
        history = {
            'train_losses': train_losses,
            'val_losses': val_losses,
            'train_accuracies': train_accuracies,
            'val_accuracies': val_accuracies
        }
        
        return model, history, training_time
    
    def evaluate_pytorch_model(model, data):
        """评估PyTorch模型"""
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)
        model.eval()
        
        test_dataset = TensorDataset(
            torch.FloatTensor(data['X_test']),
            torch.LongTensor(data['y_test'])
        )
        test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)
        
        criterion = nn.CrossEntropyLoss()
        
        total_loss = 0.0
        all_predictions = []
        all_probabilities = []
        
        with torch.no_grad():
            for batch_X, batch_y in test_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                
                total_loss += loss.item()
                probabilities = F.softmax(outputs, dim=1)
                _, predicted = torch.max(outputs, 1)
                
                all_predictions.extend(predicted.cpu().numpy())
                all_probabilities.extend(probabilities.cpu().numpy())
        
        avg_loss = total_loss / len(test_loader)
        accuracy = np.mean(np.array(all_predictions) == data['y_test'])
        
        return {
            'test_loss': avg_loss,
            'test_accuracy': accuracy,
            'y_pred': np.array(all_predictions),
            'y_pred_proba': np.array(all_probabilities)
        }
    
    # 创建和训练PyTorch模型
    print("\n=== PyTorch实现 ===")
    pytorch_model = PyTorchNN()
    print(f"PyTorch模型结构:")
    print(pytorch_model)
    
    pytorch_trained_model, pytorch_history, pytorch_training_time = train_pytorch_model(pytorch_model, data)
    pytorch_results = evaluate_pytorch_model(pytorch_trained_model, data)
    
    print(f"\nPyTorch模型结果:")
    print(f"  测试准确率: {pytorch_results['test_accuracy']:.4f}")
    print(f"  测试损失: {pytorch_results['test_loss']:.4f}")
    print(f"  训练时间: {pytorch_training_time:.2f}秒")
else:
    print("PyTorch不可用，跳过PyTorch实现")
    pytorch_model = None
    pytorch_history = None
    pytorch_results = None
    pytorch_training_time = None

## 4. 模拟标准库实现（用于对比）

In [ ]:
# 如果深度学习框架不可用，提供一个模拟的高性能实现
def create_simulated_framework_implementation(data):
    """模拟深度学习框架的高性能实现"""
    print("=== 模拟标准库实现 ===")
    
    class OptimizedNeuralNetwork:
        """优化的神经网络实现（模拟标准库性能）"""
        def __init__(self, layer_sizes=[784, 256, 128, 10]):
            self.layer_sizes = layer_sizes
            self.weights = []
            self.biases = []
            
            # 使用更好的初始化方法
            for i in range(len(layer_sizes) - 1):
                # He初始化
                limit = np.sqrt(6 / layer_sizes[i]) * 2
                W = np.random.uniform(-limit, limit, (layer_sizes[i], layer_sizes[i+1]))
                b = np.zeros((1, layer_sizes[i+1]))
                self.weights.append(W)
                self.biases.append(b)
        
        def relu(self, x):
            return np.maximum(0, x)
        
        def relu_derivative(self, x):
            return (x > 0).astype(float)
        
        def softmax(self, x):
            exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))
            return exp_x / np.sum(exp_x, axis=1, keepdims=True)
        
        def forward(self, X):
            self.activations = [X]
            self.z_values = []
            
            for i in range(len(self.weights) - 1):
                z = np.dot(self.activations[-1], self.weights[i]) + self.biases[i]
                self.z_values.append(z)
                a = self.relu(z)
                self.activations.append(a)
            
            # 输出层
            z_output = np.dot(self.activations[-1], self.weights[-1]) + self.biases[-1]
            self.z_values.append(z_output)
            output = self.softmax(z_output)
            self.activations.append(output)
            
            return output
        
        def backward(self, X, y, learning_rate=0.001):
            m = X.shape[0]
            
            # 输出层梯度
            output_error = (self.activations[-1] - y) / m
            
            d_weights = []
            d_biases = []
            
            # 输出层梯度
            dW_output = np.dot(self.activations[-2].T, output_error)
            db_output = np.sum(output_error, axis=0, keepdims=True)
            
            d_weights.insert(0, dW_output)
            d_biases.insert(0, db_output)
            
            # 反向传播误差
            error = output_error
            
            for i in range(len(self.weights) - 2, -1, -1):
                error = np.dot(error, self.weights[i+1].T) * self.relu_derivative(self.z_values[i])
                
                dW = np.dot(self.activations[i].T, error)
                db = np.sum(error, axis=0, keepdims=True)
                
                d_weights.insert(0, dW)
                d_biases.insert(0, db)
            
            # 参数更新（带权重衰减）
            weight_decay = 0.0001
            for i in range(len(self.weights)):
                self.weights[i] -= learning_rate * (d_weights[i] + weight_decay * self.weights[i])
                self.biases[i] -= learning_rate * d_biases[i]
        
        def predict(self, X):
            output = self.forward(X)
            return np.argmax(output, axis=1)
    
    # One-hot编码
    def one_hot_encode(y, num_classes=10):
        encoded = np.zeros((len(y), num_classes))
        encoded[np.arange(len(y)), y] = 1
        return encoded
    
    # 训练模型
    print("创建优化的神经网络模型...")
    model = OptimizedNeuralNetwork()
    
    # 准备数据
    y_train_encoded = one_hot_encode(data['y_train'])
    y_val_encoded = one_hot_encode(data['y_val'])
    y_test_encoded = one_hot_encode(data['y_test'])
    
    # 训练参数
    epochs = 50
    batch_size = 64
    learning_rate = 0.001
    
    print(f"开始训练 (epochs={epochs}, batch_size={batch_size}, lr={learning_rate})...")
    
    train_losses = []
    val_losses = []
    train_accuracies = []
    val_accuracies = []
    
    n_samples = len(data['X_train'])
    n_batches = n_samples // batch_size
    
    best_val_accuracy = 0
    patience = 10
    patience_counter = 0
    
    start_time = time.time()
    
    for epoch in range(epochs):
        # 打乱数据
        indices = np.random.permutation(n_samples)
        X_train_shuffled = data['X_train'][indices]
        y_train_shuffled = y_train_encoded[indices]
        
        epoch_train_loss = 0
        epoch_train_correct = 0
        
        # 批量训练
        for i in range(n_batches):
            start_idx = i * batch_size
            end_idx = (i + 1) * batch_size
            
            X_batch = X_train_shuffled[start_idx:end_idx]
            y_batch = y_train_shuffled[start_idx:end_idx]
            
            # 前向传播
            output = model.forward(X_batch)
            
            # 计算损失
            loss = -np.mean(np.sum(y_batch * np.log(output + 1e-15), axis=1))
            epoch_train_loss += loss
            
            # 计算准确率
            predictions = np.argmax(output, axis=1)
            true_labels = np.argmax(y_batch, axis=1)
            epoch_train_correct += np.sum(predictions == true_labels)
            
            # 反向传播
            model.backward(X_batch, y_batch, learning_rate)
        
        # 验证
        val_output = model.forward(data['X_val'])
        val_loss = -np.mean(np.sum(y_val_encoded * np.log(val_output + 1e-15), axis=1))
        val_predictions = np.argmax(val_output, axis=1)
        val_accuracy = np.mean(val_predictions == data['y_val'])
        
        # 计算指标
        avg_train_loss = epoch_train_loss / n_batches
        train_accuracy = epoch_train_correct / n_samples
        
        train_losses.append(avg_train_loss)
        val_losses.append(val_loss)
        train_accuracies.append(train_accuracy)
        val_accuracies.append(val_accuracy)
        
        # 早停检查
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            patience_counter = 0
            # 保存最佳模型
            best_weights = [w.copy() for w in model.weights]
            best_biases = [b.copy() for b in model.biases]
        else:
            patience_counter += 1
        
        if patience_counter >= patience:
            print(f"早停于epoch {epoch+1}")
            # 恢复最佳模型
            model.weights = best_weights
            model.biases = best_biases
            break
        
        if (epoch + 1) % 10 == 0:
            print(f"Epoch {epoch+1}/{epochs}: ")
            print(f"  训练损失: {avg_train_loss:.4f}, 训练准确率: {train_accuracy:.4f}")
            print(f"  验证损失: {val_loss:.4f}, 验证准确率: {val_accuracy:.4f}")
    
    training_time = time.time() - start_time
    print(f"训练完成，用时: {training_time:.2f}秒")
    
    # 测试评估
    test_output = model.forward(data['X_test'])
    test_loss = -np.mean(np.sum(y_test_encoded * np.log(test_output + 1e-15), axis=1))
    test_predictions = np.argmax(test_output, axis=1)
    test_accuracy = np.mean(test_predictions == data['y_test'])
    
    history = {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'train_accuracies': train_accuracies,
        'val_accuracies': val_accuracies
    }
    
    results = {
        'test_loss': test_loss,
        'test_accuracy': test_accuracy,
        'y_pred': test_predictions,
        'y_pred_proba': test_output
    }
    
    print(f"\n模拟标准库模型结果:")
    print(f"  测试准确率: {test_accuracy:.4f}")
    print(f"  测试损失: {test_loss:.4f}")
    print(f"  训练时间: {training_time:.2f}秒")
    
    return model, history, training_time, results

# 如果没有可用框架，使用模拟实现
if not TF_AVAILABLE and not PYTORCH_AVAILABLE:
    simulated_model, simulated_history, simulated_training_time, simulated_results = create_simulated_framework_implementation(data)
else:
    simulated_model = None
    simulated_history = None
    simulated_training_time = None
    simulated_results = None

## 5. 性能对比分析

In [ ]:
def compare_frameworks():
    """对比不同框架的性能"""
    print("\n" + "="*80)
    print("                    框架性能对比分析")
    print("="*80)
    
    frameworks = []
    if tf_results is not None:
        frameworks.append(('TensorFlow/Keras', tf_results, tf_history, tf_training_time))
    if pytorch_results is not None:
        frameworks.append(('PyTorch', pytorch_results, pytorch_history, pytorch_training_time))
    if simulated_results is not None:
        frameworks.append(('模拟标准库', simulated_results, simulated_history, simulated_training_time))
    
    if not frameworks:
        print("无可用的框架进行对比")
        return
    
    # 性能对比表格
    print("\n【性能对比表】")
    print(f"{'框架':<15} {'测试准确率':<12} {'测试损失':<12} {'训练时间(秒)':<12} {'相对速度':<10}")
    print("-" * 70)
    
    best_time = min(framework[3] for framework in frameworks)
    
    for name, results, history, training_time in frameworks:
        relative_speed = training_time / best_time
        print(f"{name:<15} {results['test_accuracy']:<12.4f} {results['test_loss']:<12.4f} {training_time:<12.2f} {relative_speed:<10.2f}x")
    
    # 绘制对比图表
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. 准确率对比
    ax1 = axes[0, 0]
    names = [f[0] for f in frameworks]
    accuracies = [f[1]['test_accuracy'] for f in frameworks]
    bars = ax1.bar(names, accuracies, alpha=0.7, color=['blue', 'orange', 'green'][:len(names)])
    ax1.set_title('测试准确率对比')
    ax1.set_ylabel('准确率')
    ax1.set_ylim(0.9, 1.0)
    ax1.grid(True, alpha=0.3)
    
    # 在柱子上添加数值
    for bar, acc in zip(bars, accuracies):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
                f'{acc:.4f}', ha='center', va='bottom')
    
    # 2. 训练时间对比
    ax2 = axes[0, 1]
    times = [f[3] for f in frameworks]
    bars = ax2.bar(names, times, alpha=0.7, color=['blue', 'orange', 'green'][:len(names)])
    ax2.set_title('训练时间对比')
    ax2.set_ylabel('时间 (秒)')
    ax2.grid(True, alpha=0.3)
    
    for bar, time in zip(bars, times):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + max(times)*0.01,
                f'{time:.1f}s', ha='center', va='bottom')
    
    # 3. 训练过程对比 (准确率)
    ax3 = axes[1, 0]
    for name, _, history, _ in frameworks:
        if history and 'train_accuracies' in history:
            ax3.plot(history['train_accuracies'], label=f'{name} (训练)', linewidth=2)
            ax3.plot(history['val_accuracies'], label=f'{name} (验证)', linestyle='--', linewidth=2)
    ax3.set_title('训练过程准确率变化')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('准确率')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # 4. 训练过程对比 (损失)
    ax4 = axes[1, 1]
    for name, _, history, _ in frameworks:
        if history and 'train_losses' in history:
            ax4.plot(history['train_losses'], label=f'{name} (训练)', linewidth=2)
            ax4.plot(history['val_losses'], label=f'{name} (验证)', linestyle='--', linewidth=2)
    ax4.set_title('训练过程损失变化')
    ax4.set_xlabel('Epoch')
    ax4.set_ylabel('损失')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 详细分析报告
    print("\n【详细分析报告】")
    
    # 找出最佳准确率
    best_accuracy = max(frameworks, key=lambda x: x[1]['test_accuracy'])
    print(f"\n最高准确率: {best_accuracy[0]} - {best_accuracy[1]['test_accuracy']:.4f}")
    
    # 找出最快训练时间
    fastest = min(frameworks, key=lambda x: x[3])
    print(f"最快训练: {fastest[0]} - {fastest[3]:.2f}秒")
    
    # 收敛性分析
    print(f"\n【收敛性分析】")
    for name, _, history, _ in frameworks:
        if history and 'val_accuracies' in history:
            final_acc = history['val_accuracies'][-1]
            max_acc = max(history['val_accuracies'])
            converged_epoch = next((i for i, acc in enumerate(history['val_accuracies']) 
                                 if acc > max_acc * 0.99), len(history['val_accuracies']) - 1)
            print(f"  {name}: 最终准确率={final_acc:.4f}, 最高准确率={max_acc:.4f}, 收敛epoch={converged_epoch+1}")
    
    return frameworks

# 执行对比分析
frameworks_comparison = compare_frameworks()

## 6. 混淆矩阵对比分析

In [ ]:
def compare_confusion_matrices():
    """对比不同框架的混淆矩阵"""
    frameworks_data = []
    
    if tf_results is not None:
        frameworks_data.append(('TensorFlow/Keras', tf_results['y_pred'], data['y_test']))
    if pytorch_results is not None:
        frameworks_data.append(('PyTorch', pytorch_results['y_pred'], data['y_test']))
    if simulated_results is not None:
        frameworks_data.append(('模拟标准库', simulated_results['y_pred'], data['y_test']))
    
    if not frameworks_data:
        print("无可用的框架进行混淆矩阵对比")
        return
    
    n_frameworks = len(frameworks_data)
    fig, axes = plt.subplots(1, n_frameworks, figsize=(6*n_frameworks, 5))
    
    if n_frameworks == 1:
        axes = [axes]
    
    for i, (name, y_pred, y_true) in enumerate(frameworks_data):
        cm = confusion_matrix(y_true, y_pred)
        
        # 计算每个类别的准确率
        class_accuracies = cm.diagonal() / cm.sum(axis=1)
        
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i],
                   xticklabels=range(10), yticklabels=range(10))
        axes[i].set_title(f'{name}\n平均准确率: {np.mean(class_accuracies):.4f}')
        axes[i].set_xlabel('预测标签')
        axes[i].set_ylabel('真实标签')
    
    plt.tight_layout()
    plt.show()
    
    # 类别级别对比
    print("\n【类别级别准确率对比】")
    print(f"{'数字':<6}", end="")
    for name, _, _ in frameworks_data:
        print(f"{name:<15}", end="")
    print()
    print("-" * (6 + 15 * len(frameworks_data)))
    
    for digit in range(10):
        print(f"{digit:<6}", end="")
        for name, y_pred, y_true in frameworks_data:
            mask = y_true == digit
            if mask.sum() > 0:
                acc = np.mean(y_pred[mask] == digit)
                print(f"{acc:<15.4f}", end="")
            else:
                print(f"{'N/A':<15}", end="")
        print()

# 混淆矩阵对比
compare_confusion_matrices()

## 7. 错误案例分析对比

In [ ]:
def compare_error_analysis():
    """对比不同框架的错误分析"""
    frameworks_data = []
    
    if tf_results is not None:
        frameworks_data.append(('TensorFlow/Keras', tf_results['y_pred'], data['y_test']))
    if pytorch_results is not None:
        frameworks_data.append(('PyTorch', pytorch_results['y_pred'], data['y_test']))
    if simulated_results is not None:
        frameworks_data.append(('模拟标准库', simulated_results['y_pred'], data['y_test']))
    
    if not frameworks_data:
        print("无可用的框架进行错误分析对比")
        return
    
    print("\n【错误分析对比】")
    
    # 错误率对比
    print(f"{'框架':<15} {'总错误数':<10} {'错误率':<10} {'主要错误类型':<20}")
    print("-" * 65)
    
    for name, y_pred, y_true in frameworks_data:
        errors = np.where(y_pred != y_true)[0]
        error_rate = len(errors) / len(y_true)
        
        # 分析最常见的错误类型
        error_pairs = {}
        for idx in errors:
            true_label = y_true[idx]
            pred_label = y_pred[idx]
            pair = (true_label, pred_label)
            error_pairs[pair] = error_pairs.get(pair, 0) + 1
        
        # 找出最常见的错误
        if error_pairs:
            most_common_error = max(error_pairs.items(), key=lambda x: x[1])
            main_error_type = f"{most_common_error[0][0]}→{most_common_error[0][1]} ({most_common_error[1]}次)"
        else:
            main_error_type = "无错误"
        
        print(f"{name:<15} {len(errors):<10} {error_rate:<10.4f} {main_error_type:<20}")
    
    # 可视化错误样本对比
    fig, axes = plt.subplots(len(frameworks_data), 5, figsize=(15, 3*len(frameworks_data)))
    
    if len(frameworks_data) == 1:
        axes = axes.reshape(1, -1)
    
    for i, (name, y_pred, y_true) in enumerate(frameworks_data):
        # 找到错误分类的样本
        errors = np.where(y_pred != y_true)[0]
        
        if len(errors) > 0:
            # 随机选择5个错误样本
            selected_errors = np.random.choice(errors, min(5, len(errors)), replace=False)
            
            for j, idx in enumerate(selected_errors):
                axes[i, j].imshow(data['X_test'][idx].reshape(28, 28), cmap='gray')
                axes[i, j].set_title(f'{name}\n真实: {y_true[idx]}, 预测: {y_pred[idx]}')
                axes[i, j].axis('off')
        else:
            # 如果没有错误，显示一些正确的样本
            correct = np.where(y_pred == y_true)[0]
            selected_correct = np.random.choice(correct, min(5, len(correct)), replace=False)
            
            for j, idx in enumerate(selected_correct):
                axes[i, j].imshow(data['X_test'][idx].reshape(28, 28), cmap='gray')
                axes[i, j].set_title(f'{name}\n正确: {y_true[idx]}")
                axes[i, j].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # 共同错误分析
    print("\n【共同错误分析】")
    if len(frameworks_data) >= 2:
        # 找出所有框架都分类错误的样本
        all_errors = []
        for _, y_pred, y_true in frameworks_data:
            errors = set(np.where(y_pred != y_true)[0])
            all_errors.append(errors)
        
        common_errors = set.intersection(*all_errors)
        print(f"所有框架都分类错误的样本数: {len(common_errors)}")
        print(f"占总测试样本的比例: {len(common_errors)/len(data['y_test']):.4f}")
        
        # 分析共同错误的原因
        if len(common_errors) > 0:
            common_errors_list = list(common_errors)
            common_true_labels = data['y_test'][common_errors_list]
            
            print("\n共同错误样本的标签分布:")
            for digit in range(10):
                count = np.sum(common_true_labels == digit)
                if count > 0:
                    print(f"  数字 {digit}: {count} 个样本")
            
            # 显示一些共同错误的样本
            if len(common_errors) > 0:
                sample_errors = list(common_errors)[:min(5, len(common_errors))]
                
                fig, axes = plt.subplots(1, len(sample_errors), figsize=(15, 3))
                if len(sample_errors) == 1:
                    axes = [axes]
                
                for i, idx in enumerate(sample_errors):
                    axes[i].imshow(data['X_test'][idx].reshape(28, 28), cmap='gray')
                    true_label = data['y_test'][idx]
                    
                    # 收集所有框架的预测
                    predictions = []
                    for name, y_pred, _ in frameworks_data:
                        predictions.append(f"{name}: {y_pred[idx]}")
                    
                    axes[i].set_title(f'真实: {true_label}\n' + '\n'.join(predictions))
                    axes[i].axis('off')
                
                plt.tight_layout()
                plt.show()

# 错误分析对比
compare_error_analysis()

## 8. 框架特性对比

In [ ]:
def compare_framework_features():
    """对比不同框架的特性"""
    print("\n" + "="*80)
    print("                   框架特性详细对比")
    print("="*80)
    
    features_comparison = [
        ("易用性", {
            "手写实现": "需要手动实现所有算法，学习成本高",
            "TensorFlow": "API简洁，文档丰富，易于上手",
            "PyTorch": "Pythonic风格，动态图，调试方便"
        }),
        ("性能", {
            "手写实现": "纯Python实现，性能较慢",
            "TensorFlow": "高度优化，支持GPU加速",
            "PyTorch": "优秀性能，GPU加速良好"
        }),
        ("灵活性", {
            "手写实现": "完全可定制，适合学习和研究",
            "TensorFlow": "模块化设计，自定义层支持良好",
            "PyTorch": "极高的灵活性，研究首选"
        }),
        ("调试", {
            "手写实现": "可完全控制，便于理解内部机制",
            "TensorFlow": "TensorBoard可视化，调试工具完善",
            "PyTorch": "动态图便于调试，错误信息清晰"
        }),
        ("生态系统", {
            "手写实现": "无依赖，可移植性强",
            "TensorFlow": "丰富生态系统，部署工具完善",
            "PyTorch": "快速发展，社区活跃"
        }),
        ("生产部署", {
            "手写实现": "需要大量优化工作",
            "TensorFlow": "TensorFlow Serving等部署方案成熟",
            "PyTorch": "TorchScript等部署方案不断完善"
        })
    ]
    
    for feature, comparisons in features_comparison:
        print(f"\n【{feature}】")
        for framework, description in comparisons.items():
            # 检查相关框架是否可用
            if framework == "TensorFlow" and not TF_AVAILABLE:
                continue
            if framework == "PyTorch" and not PYTORCH_AVAILABLE:
                continue
            print(f"  {framework}: {description}")
    
    # 推荐使用场景
    print("\n【推荐使用场景】")
    recommendations = [
        ("学习理解", "手写实现", "适合深入理解神经网络原理"),
        ("快速原型", "PyTorch", "Pythonic风格，开发效率高"),
        ("生产部署", "TensorFlow", "部署工具完善，工业界首选"),
        ("学术研究", "PyTorch", "灵活性高，社区活跃"),
        ("移动部署", "TensorFlow Lite", "移动端优化方案成熟")
    ]
    
    for scenario, framework, reason in recommendations:
        print(f"  {scenario}: {framework} - {reason}")
    
    # 性能优化建议
    print("\n【性能优化建议】")
    optimization_tips = [
        "1. 批量处理: 使用合适的批量大小提升GPU利用率",
        "2. 数据预处理: 预处理和加载并行化",
        "3. 模型优化: 使用更高效的网络结构",
        "4. 学习率调度: 使用学习率衰减策略",
        "5. 正则化: 防止过拟合，提升泛化能力",
        "6. 早停机制: 避免过拟合，节省训练时间",
        "7. 模型检查点: 定期保存最佳模型",
        "8. 分布式训练: 大规模数据集使用多GPU训练"
    ]
    
    for tip in optimization_tips:
        print(f"  {tip}")
    
    # 框架选择决策树
    print("\n【框架选择决策树】")
    print("  你的主要目标是什么？")
    print("  ├─ 学习原理 → 手写实现")
    print("  ├─ 快速开发")
    print("  │  ├─ 研究实验 → PyTorch")
    print("  │  └─ 工业应用 → TensorFlow")
    print("  └─ 生产部署")
    print("     ├─ 移动端 → TensorFlow Lite")
    print("     ├─ Web端 → TensorFlow.js")
    print("     └─ 服务器 → TensorFlow Serving / PyTorch Serve")

# 特性对比
compare_framework_features()

## 9. 总结和建议

In [ ]:
def final_summary_and_recommendations():
    """最终总结和建议"""
    print("\n" + "="*80)
    print("              标准库对比实现 - 最终总结报告")
    print("="*80)
    
    print("\n【实验成果】")
    print("1. 成功实现了多种框架的神经网络数字识别")
    print("2. 对比了不同框架的性能、易用性和特点")
    print("3. 分析了框架间的差异和适用场景")
    print("4. 提供了框架选择的实用建议")
    
    # 统计可用框架
    available_frameworks = []
    if TF_AVAILABLE:
        available_frameworks.append("TensorFlow")
    if PYTORCH_AVAILABLE:
        available_frameworks.append("PyTorch")
    if simulated_results is not None:
        available_frameworks.append("模拟实现")
    
    print(f"\n【本实验测试的框架】")
    for framework in available_frameworks:
        print(f"  ✓ {framework}")
    
    print("\n【主要发现】")
    print("1. 性能差异: 标准库通常比手写实现快10-100倍")
    print("2. 准确率相近: 相同架构下不同框架准确率差异很小")
    print("3. 开发效率: 标准库大幅提升开发和调试效率")
    print("4. 学习价值: 手写实现对理解原理很有帮助")
    
    print("\n【最佳实践建议】")
    print("1. 学习阶段: 先手写实现理解原理，再使用标准库")
    print("2. 项目开发: 根据需求选择合适的框架")
    print("3. 性能优化: 充分利用框架提供的优化功能")
    print("4. 调试技巧: 利用框架的可视化和调试工具")
    
    print("\n【进阶学习方向】")
    advanced_topics = [
        "1. 卷积神经网络(CNN): 专门用于图像处理",
        "2. 循环神经网络(RNN): 适合序列数据处理",
        "3. 注意力机制和Transformer: 当前最先进的架构",
        "4. 迁移学习: 利用预训练模型提升性能",
        "5. 模型压缩和量化: 部署到移动和边缘设备",
        "6. 自动化机器学习: 自动搜索最佳架构和超参数",
        "7. 模型解释性: 理解模型决策过程",
        "8. 联邦学习: 隐私保护的分布式学习"
    ]
    
    for topic in advanced_topics:
        print(f"  {topic}")
    
    print("\n【实用工具推荐】")
    tools = [
        "• TensorBoard: TensorFlow可视化工具",
        "• Weights & Biases: 实验跟踪和可视化",
        "• MLflow: 机器学习生命周期管理",
        "• Neptune.ai: 实验管理和协作平台",
        "• Comet.ml: 模型监控和版本控制"
    ]
    
    for tool in tools:
        print(f"  {tool}")
    
    print("\n" + "="*80)
    print("感谢使用神经网络数字识别项目！")
    print("希望这个对比实验能帮助你选择合适的深度学习框架。")
    print("="*80)

# 显示最终总结
final_summary_and_recommendations()

print("\n标准库对比实现notebook已完成！")
print("主要成果:")
print("1. TensorFlow/Keras实现对比")
print("2. PyTorch实现对比")
print("3. 性能和准确率对比分析")
print("4. 混淆矩阵和错误分析对比")
print("5. 框架特性和使用场景分析")
print("6. 实用的选择建议和最佳实践")